In [1]:
import h5py
import cv2
import os, os.path
import pandas as pd
import mahotas as mt
import numpy as np

from ast import literal_eval
from tkinter import *
from PIL import Image ,ImageTk
from tkinter import filedialog
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neural_network import MLPClassifier

In [2]:
def select_image():
    # grab a reference to the image panels
    global panel
    # open a file chooser dialog and allow the user to select an input
    # image
    W = 410
    path = filedialog.askopenfilename()
    
    if len(path) > 0:
        
        image = cv2.imread(path)
        statusText.set(path)
        label = predictLabel(image,W)
        labelText.set(label)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        height, width, depth = image.shape
        imgScale = W/width
        newX,newY = image.shape[1]*imgScale, image.shape[0]*imgScale
        image = cv2.resize(image,(int(newX),int(newY)))

        image = Image.fromarray(image)
        image = ImageTk.PhotoImage(image)
        
    if panel is None:
        panel = Label(image=image)
        panel.image = image
        panel.pack(padx=10, pady=10)

    else:
        # update the pannels
        panel.configure(image=image)
        panel.image = image

In [3]:
def predictLabel(image,W):
    target= ['banana', 'blueberry', 'cherry', 'chinese_pear', 
             'dragon_fruit','green_apple', 'green_grape', 'guava', 'kiwi', 'longan', 'mango',
             'marian_plum', 'orange', 'passion_fruit', 'rambutan', 'red_apple',
             'rose_apple', 'salacca', 'strawberry', 'tamarind']
    
    height, width, depth = image.shape
    imgScale = W/width
    newX,newY = image.shape[1]*imgScale, image.shape[0]*imgScale
    image = cv2.resize(image,(int(newX),int(newY)))
    
    haralick = extract_haralick_features(image)
    lbp = extract_lbp_features(image)
    hsv = extract_histogram_features(image)
    cld = extract_cld_features(image)
     
    x = np.hstack([haralick,lbp,hsv,cld])
    x_train , y_train = train()

    mlp = MLPClassifier(solver='lbfgs',alpha=0.1,random_state=9,epsilon=1e-04,hidden_layer_sizes=100,activation='logistic')
    mlp.fit(x_train,y_train)
    prediction = mlp.predict(x.reshape(1, -1))[0]
    prediction = str(target[prediction]).replace("_", " ").capitalize()
    return prediction

In [4]:
def train():
    h5f_data = h5py.File('./output/data20-1000.h5', 'r')
    h5f_label = h5py.File('./output/labels.h5', 'r')
    
    global_features_string = h5f_data['glcm_lbp_hsv_cld']
    global_labels_string = h5f_label['labels']
    

    x_train = np.array(global_features_string)
    y_train = np.array(global_labels_string)
    h5f_data.close()
    h5f_label.close()

    return x_train, y_train

In [ ]:
def extract_haralick_features(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    # calculate haralick texture features for 4 types of adjacency
    haralick = mt.features.haralick(image).mean(axis=0)
    # take the mean of it and return it
    return haralick

def extract_lbp_features(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    # calculate lbp texture features (image, radius, points, ignore_zeros=False)
    lbp_textures = mt.features.lbp(image, 8, 8, False)
    return lbp_textures

def extract_histogram_features(image):
#     image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hHistr = list(cv2.calcHist([image],[0],None,[32],[0,180]).flatten())
    sHistr = list(cv2.calcHist([image],[1],None,[16],[0,256]).flatten())
    vHistr = list(cv2.calcHist([image],[2],None,[16],[0,256]).flatten())
    hist = np.concatenate((hHistr,sHistr,vHistr))
    return np.array(hist)

In [ ]:
def extract_cld_features(image):
#     image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    cld_rows = 4
    cld_cols = 4
    averages = np.zeros((cld_rows,cld_cols,3))
    imgH, imgW, _ = image.shape
    for row in range(cld_rows):
        for col in range(cld_cols):   
            slice = image[int(imgH/cld_rows * row): int(imgH/cld_rows * (row+1)),int(imgW/cld_cols*col) : int(imgW/cld_cols*(col+1))]
            average_color_per_row = np.mean(slice, axis=0)
            average_color = np.mean(average_color_per_row, axis=0)
            average_color = np.uint8(average_color)
            averages[row][col][0] = average_color[0]
            averages[row][col][1] = average_color[1]
            averages[row][col][2] = average_color[2]
    icon = cv2.cvtColor(np.array(averages, dtype=np.uint8), cv2.COLOR_BGR2YCR_CB)
    y, cr, cb = cv2.split(icon)
    dct_y = cv2.dct(np.float32(y))
    dct_cb = cv2.dct(np.float32(cb))
    dct_cr = cv2.dct(np.float32(cr))
    dct_y_zigzag = []
    dct_cb_zigzag = []
    dct_cr_zigzag = []
    flip = True
    flipped_dct_y = np.fliplr(dct_y)
    flipped_dct_cb = np.fliplr(dct_cb)
    flipped_dct_cr = np.fliplr(dct_cr)
    for i in range(cld_rows + cld_cols -1):
        k_diag = cld_rows - 1 - i
        diag_y = np.diag(flipped_dct_y, k=k_diag)
        diag_cb = np.diag(flipped_dct_cb, k=k_diag)
        diag_cr = np.diag(flipped_dct_cr, k=k_diag)
        if flip:
            diag_y = diag_y[::-1]
            diag_cb = diag_cb[::-1]
            diag_cr = diag_cr[::-1]
        dct_y_zigzag.append(diag_y)
        dct_cb_zigzag.append(diag_cb)
        dct_cr_zigzag.append(diag_cr)
        flip = not flip
    return np.concatenate([np.concatenate(dct_y_zigzag), np.concatenate(dct_cb_zigzag), np.concatenate(dct_cr_zigzag)])

In [ ]:
win = Tk()
panel = None
win.option_add('*font', ('tohama', 10))
# create a button, then when pressed, will trigger a file chooser
# dialog and allow the user to select an input image; then add the
# button the GUI
statusText = StringVar(win)
labelText = StringVar(win)
statusText.set("Choose image")

win.title("FIB classification program")
win.resizable(False, False)  # This code helps to disable windows from resizing

window_height = 550
window_width = 500


path = Entry(win, textvariable= statusText,bg='white',justify='center')
path.pack(side='top',padx=10,pady=5,fill="x",anchor="center")
btn = Button(win, text="Browse...", command=select_image)
btn.pack(side='top',padx=10,pady=5,fill="x")
R1 = Entry(win, textvariable= labelText,justify='center')
R1.pack(side='bottom',padx=10,pady=10)


# main = Tk() Label(main, text = "Enter Num 1:")
# Label(main, text = "Enter Num 2:").grid(row=1) 
# Label(main, text = "The Sum is:").grid(row=2)

screen_width = win.winfo_screenwidth()
screen_height = win.winfo_screenheight()
x_cordinate = int((screen_width/2) - (window_width/2))
y_cordinate = int((screen_height/2) - (window_height/2))

win.geometry("{}x{}+{}+{}".format(window_width, window_height, x_cordinate, y_cordinate))
win.mainloop()